In [1]:
# Load Library
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from getME import getME # Local file 

## Data Pre-processing

In [2]:
# Read the sample data
data = pd.read_csv('data.csv' )

In [3]:
# Take a look at all the features
data.columns

Index(['metroarea', 'carcount', 'modetowork', 'travelburden', 'transitgood',
       'tripcount', 'tripmile', 'distancetowork', 'transituse', 'hhincome',
       'hhsize', 'childage', 'houseown', 'urbansize', 'popdensity', 'gasprice',
       'age', 'gender', 'education', 'workschoolretire', 'hhstate',
       'Walk.Index_Den', 'Walk.Index_Pop', 'Transit.Score', 'poverty_pop',
       'disability_pop', 'bachelor_pop', 'employ_workforce'],
      dtype='object')

In [4]:
# Select the features we are interested in
# Note: before this step, we need to check the correlations, corlinearity, what are our study objectives, etc...
selected_variables = ['carcount', 'modetowork',
                      'tripcount','tripmile',
                      'hhsize','hhincome',
                      'gender','age','education','workschoolretire',
                      'urbansize','Transit.Score','Walk.Index_Pop',
                      'travelburden']
model_data = data[selected_variables]

In [5]:
# Descriptive statistical - before encoding
model_data.describe()

,carcount,modetowork,tripcount,tripmile,hhsize,hhincome,gender,age,education,workschoolretire,urbansize,Transit.Score,Walk.Index_Pop,travelburden
count,10141.000000,10141.000000,10141.000000,10141.000000,10141.000000,10141.000000,10141.000000,10141.000000,10141.000000,10141.000000,10141.000000,10141.000000,10141.000000,10141.000000
mean,1.054433,3.038655,3.979785,36.657541,2.606646,3.170595,1.490386,45.099694,3.746869,10.361108,4.414555,3.544552,10.383681,2.009072
std,0.552380,0.485779,2.571519,53.071743,1.183546,0.996289,0.499932,14.117366,1.061576,2.016637,1.001045,1.631513,1.695027,0.823901
min,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,18.000000,1.000000,4.000000,1.000000,0.300000,6.204787,1.000000
25%,1.000000,3.000000,2.000000,9.814000,2.000000,3.000000,1.000000,33.000000,3.000000,11.000000,4.000000,2.400000,9.369801,1.000000
50%,1.000000,3.000000,4.000000,23.791000,2.000000,3.000000,1.000000,45.000000,4.000000,11.000000,4.000000,3.400000,10.817957,2.000000
75%,1.000000,3.000000,5.000000,44.957000,4.000000,4.000000,2.000000,56.000000,5.000000,11.000000,5.000000,4.600000,11.869645,3.000000
max,3.000000,5.000000,47.000000,1023.582000,5.000000,4.000000,2.000000,92.000000,5.000000,12.000000,6.000000,9.300000,13.017643,3.000000


### Variable Re-coding

In [6]:
# We should care about how to set up the baseline
model_data = model_data.copy()
# car counts: 0, 1 and more than 1
model_data.loc[model_data['carcount'] > 0,'carcount'] = 1
# mode to work: 0 for car, 1 for others
model_data.loc[model_data['modetowork'] == 3,'modetowork'] = 0
model_data.loc[model_data['modetowork'] > 0,'modetowork'] = 1
# trip counts: 0, 1, 2, 3 and more than 3
model_data.loc[model_data['tripcount'] > 3,'tripcount'] = 3
# hhincome: 0-2.5k, 2.5k-75k, over 75k
model_data.loc[model_data['hhincome'] < 3,'hhincome'] = 1
model_data.loc[model_data['hhincome'] == 3,'hhincome'] = 2
model_data.loc[model_data['hhincome'] == 4 ,'hhincome'] = 3
# education: highschool and below, college, graduate
model_data.loc[model_data['education'] < 3,'education'] = 1
model_data.loc[model_data['education'] == 3,'education'] = 2
model_data.loc[model_data['education'] > 2,'education'] = 3
# urbansize: below 1 m, 1 m no rail, 1 m rail
model_data.loc[(model_data['urbansize'] < 4) | (model_data['urbansize'] ==6) ,'urbansize'] = 1
model_data.loc[model_data['urbansize'] == 4,'urbansize'] = 2
model_data.loc[model_data['urbansize'] == 5,'urbansize'] = 3
# workschoolretire: work, other
model_data.loc[model_data['workschoolretire'] == 11,'workschoolretire'] = 0
model_data.loc[model_data['workschoolretire'] > 0,'workschoolretire'] = 1

In [7]:
# tripmiles: 0-, 1 m no rail, 1 m rail
model_data.loc[(model_data['urbansize'] < 4) | (model_data['urbansize'] ==6) ,'urbansize'] = 1
model_data.loc[model_data['urbansize'] == 4,'urbansize'] = 2
model_data.loc[model_data['urbansize'] == 5,'urbansize'] = 3

In [8]:
# Descriptive statistical - after encoding
model_data.describe()

,carcount,modetowork,tripcount,tripmile,hhsize,hhincome,gender,age,education,workschoolretire,urbansize,Transit.Score,Walk.Index_Pop,travelburden
count,10141.000000,10141.000000,10141.000000,10141.000000,10141.000000,10141.000000,10141.000000,10141.000000,10141.000000,10141.000000,10141.0,10141.000000,10141.000000,10141.000000
mean,0.898925,0.143378,2.506558,36.657541,2.606646,2.284094,1.490386,45.099694,2.467410,0.091608,1.0,3.544552,10.383681,2.009072
std,0.301443,0.350476,0.834820,53.071743,1.183546,0.774885,0.499932,14.117366,0.732621,0.288486,0.0,1.631513,1.695027,0.823901
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,18.000000,1.000000,0.000000,1.0,0.300000,6.204787,1.000000
25%,1.000000,0.000000,2.000000,9.814000,2.000000,2.000000,1.000000,33.000000,2.000000,0.000000,1.0,2.400000,9.369801,1.000000
50%,1.000000,0.000000,3.000000,23.791000,2.000000,2.000000,1.000000,45.000000,3.000000,0.000000,1.0,3.400000,10.817957,2.000000
75%,1.000000,0.000000,3.000000,44.957000,4.000000,3.000000,2.000000,56.000000,3.000000,0.000000,1.0,4.600000,11.869645,3.000000
max,1.000000,1.000000,3.000000,1023.582000,5.000000,3.000000,2.000000,92.000000,3.000000,1.000000,1.0,9.300000,13.017643,3.000000


### Prepare Dummy Variables

In [9]:
# first dependent variable
FirstDepVar = ['carcount']
# second dependent variable
SecondDepVar = ['modetowork']
# all independent variables
IndVar = ['tripcount','tripmile',
          'hhsize','hhincome',
          'gender','age','education','workschoolretire',
          'urbansize','Transit.Score','Walk.Index_Pop',
          'travelburden']
# continuous variables
ConVar=['tripmile','age','Transit.Score','Walk.Index_Pop']
# dummy variables
RegOrdVar = ['gender','education','workschoolretire',
          'urbansize','travelburden']
# ordered categorical variables
OrdCatVar = ['tripcount','hhsize','hhincome']

#### Create a dictionary to store all possible values for each variable

In [10]:
col_to_var={}
# for col_name in DummyVar+ConVar+FirstDepVar+SecondDepVar:
#     col_to_var[col_name]= model_data[col_name].value_counts().keys().tolist()
for col_name in IndVar+FirstDepVar+SecondDepVar:
    col_to_var[col_name]= model_data[col_name].value_counts().keys().tolist()

#### Create a dictionary to store all base values for each variable
##### The base of the continous variables are set to be mean value in this demo

In [11]:
col_to_var_base={}
col_to_var_base={
                 # dependent variables
                 'carcount': 0, 
                 'modetowork': 1,
    
                 # ordered categorical variables
                 'tripcount': 2,
                 'hhsize':3,
                 'hhincome':2,
    
                 # dummy variables                
                 'gender':1,
                 'education':2,
                 'workschoolretire':0,
                 'urbansize':1,
                 'travelburden': 2,
    
                 # continuous variables                 
                 'tripmile': np.mean(model_data['tripmile']),
                 'age': np.mean(model_data['age']),
                 'Transit.Score': np.mean(model_data['Transit.Score']),
                 'Walk.Index_Pop': np.mean(model_data['Walk.Index_Pop'])
                }
# col_to_var_base

#### Drop out NA values

In [12]:
model_data= model_data.dropna()
# model_data

#### Set up variables for modeling

In [13]:
dummyVar = RegOrdVar + OrdCatVar + FirstDepVar
DepVar = SecondDepVar
model_name = 'Boost'

In [14]:
# Go to see getME.py
# funtion getME
#{ model_name,
#  model_data,
#  dummyVar(categorical variables),
#  ConVar(continuous variables), 
#  DepVar(Y2 in path analysis),
#  col_to_var(the dictionary of all the variables),
#  col_to_var_base(the dictionary of all the base variables),
#  test_size = 0.2 (train_test split size, 0.2 means 20% test data and 80% train data)
#  var_marginal_dataframe:
        # First column (variable name @ specific category)
        # Var base (baseline of the variable)
        # var specific (the specific category when want to calculate the marginal effect)
        # 0 (the marginal effec of Y2 change from 0 to 1)
        # 1 (the marginal effec of Y2 change from 0 to 1)
#  base_to_Y_probablity:
        # First column (variable name @ specific category)
        # Var base (baseline of the variable)
        # var specific (the specific category when want to calculate the marginal effect)
        # 0 (the predict result at base line)
        # 1 (the predict result at specific value)
#}
var_marginal_dataframe, base_to_Y_probability =getME(model_name,model_data,
                                                      dummyVar,ConVar,DepVar,
                                                      col_to_var,col_to_var_base,
                                                      test_size = 0.2)

delete gender_1
delete education_2
delete workschoolretire_0
delete urbansize_1
delete travelburden_2
delete tripcount_2
delete hhsize_3
delete hhincome_2
delete carcount_0
X+Y1=Y2 Time =
0.731177568435669
Boost Accuracy: 0.917693445046821
[0, 1]
tripcount
hhsize
hhincome
gender
education
workschoolretire
urbansize
travelburden
carcount
modetowork
tripmile
age
Transit.Score
Walk.Index_Pop
tripmile [0.1161832176031105, -0.1161832176031239]
age [0.0033380598619154305, -0.003338059861915607]
Transit.Score [-0.010197280736589489, 0.010197280736592075]
Walk.Index_Pop [-0.030659736290894316, 0.030659736290892154]
gender 2
education 3
education 1
workschoolretire 1
travelburden 3
travelburden 1
tripcount 3
tripcount 0
tripcount 1
hhsize 2
hhsize 4
hhsize 1
hhsize 5
hhincome 3
hhincome 1
carcount 1


In [15]:
# First column (variable name @ specific category)
# Var base (baseline of the variable)
# var specific (the specific category when want to calculate the marginal effect)
# 0 (the predict result at base line for Y2 = 0)
# 1 (the predict result at base line for Y2 = 1)
base_to_Y_probability

,Var base,Var specific,0,1
tripcount@3,2.000000,3,0.506735,0.493265
tripcount@0,2.000000,0,0.506735,0.493265
tripcount@1,2.000000,1,0.506735,0.493265
hhsize@2,3.000000,2,0.506809,0.493191
hhsize@4,3.000000,4,0.506809,0.493191
hhsize@1,3.000000,1,0.506809,0.493191
hhsize@5,3.000000,5,0.506809,0.493191
hhincome@3,2.000000,3,0.507002,0.492998
hhincome@1,2.000000,1,0.507002,0.492998
gender@2,1.000000,2,0.506316,0.493684


In [16]:
# Name the columns
# "Var base"
# "Var specific"
# "car" (this is the marginal effect of the base for Y1, change from base to the other value)
# "other_modes" (this is the marginal effect of the other value of Y1, change from the other value to base)
var_marginal_dataframe.columns=["Var base","Var specific",
                                'car','other_modes']
# "Var base"
# "Var specific"
# "U-car" (this is the predict probablity of base line)
# "U-other_modes" (this is the prediction probality of the other modes, if Y1 is binary, the sum of U-car and U-other_modes should be 1)
base_to_Y_probability.columns=["Var base","Var specific",
                               "U-"+'car',"U-"+'other_modes']
direct_impacts_on_Y2=pd.concat([var_marginal_dataframe, base_to_Y_probability ],axis=1)

direct_impacts_on_Y2.to_csv(model_name+"direct_impacts_on_Y2"+ ".csv")

## Y1 model

In [17]:
dummyVar = RegOrdVar + OrdCatVar
DepVar = FirstDepVar

In [18]:
var_marginal_dataframe, base_to_Y_probability =getME(model_name,model_data,
                                                      dummyVar,ConVar,DepVar,
                                                      col_to_var,col_to_var_base,
                                                      test_size = 0.2)

delete gender_1
delete education_2
delete workschoolretire_0
delete urbansize_1
delete travelburden_2
delete tripcount_2
delete hhsize_3
delete hhincome_2
X+Y1=Y2 Time =
0.7249984741210938
Boost Accuracy: 0.9068506653523903
[0, 1]
tripcount
hhsize
hhincome
gender
education
workschoolretire
urbansize
travelburden
carcount
modetowork
tripmile
age
Transit.Score
Walk.Index_Pop
tripmile [-0.6905543400798978, 0.6905543400799157]
age [-0.004802759283960062, 0.004802759283960034]
Transit.Score [0.03180501441303086, -0.031805014413030026]
Walk.Index_Pop [0.028719088324593283, -0.028719088324591482]
gender 2
education 3
education 1
workschoolretire 1
travelburden 3
travelburden 1
tripcount 3
tripcount 0
tripcount 1
hhsize 2
hhsize 4
hhsize 1
hhsize 5
hhincome 3
hhincome 1


In [19]:
# Name the columns
# "Var base"
# "Var specific"
# "nocar" (this is the marginal effect of the base for Y1, change from base to the other value)
# "other_modes" (this is the marginal effect of the other value of Y1, change from the other value to base)
var_marginal_dataframe.columns=["Var base","Var specific",
                                'nocar','havecar']

base_to_Y_probability.columns=["Var base","Var specific",
                               "U-"+'nocar',"U-"+'havecar']
direct_impacts_on_Y1=pd.concat([var_marginal_dataframe, base_to_Y_probability ],axis=1)

direct_impacts_on_Y1.to_csv(model_name+"direct_impacts_on_Y1"+ ".csv")

In [20]:
direct_impacts_on_Y1

,Var base,Var specific,nocar,havecar,Var base,Var specific,U-nocar,U-havecar
tripcount@3,2.000000,3,0.0,0.0,2.000000,3,0.492626,0.507374
tripcount@0,2.000000,0,0.0,0.0,2.000000,0,0.492626,0.507374
tripcount@1,2.000000,1,0.0,0.0,2.000000,1,0.492626,0.507374
hhsize@2,3.000000,2,0.000702,-0.000702,3.000000,2,0.492402,0.507598
hhsize@4,3.000000,4,-0.000549,0.000549,3.000000,4,0.492402,0.507598
hhsize@1,3.000000,1,0.0,0.0,3.000000,1,0.492402,0.507598
hhsize@5,3.000000,5,0.00037,-0.00037,3.000000,5,0.492402,0.507598
hhincome@3,2.000000,3,-0.000394,0.000394,2.000000,3,0.492455,0.507545
hhincome@1,2.000000,1,0.001906,-0.001906,2.000000,1,0.492455,0.507545
gender@2,1.000000,2,0.000261,-0.000261,1.000000,2,0.492499,0.507501


In [21]:
path_ana_value=["DirectMEonY2","uxForY1","MEonY1","uY1ForY2","MEofY1onY2","IndirectMEonY2","TotalMEonY2"]
Path_Analysis_table=pd.DataFrame("",columns=["Var base","Var specific"]+path_ana_value,
                                 index=direct_impacts_on_Y2.index.tolist())

In [22]:
Path_Analysis_table["Var base"]=direct_impacts_on_Y2["Var base"].iloc[:,0]
Path_Analysis_table["Var specific"]=direct_impacts_on_Y2["Var specific"].iloc[:,0]

In [23]:
# 'car' would be changed to Y2's non-base value name
Path_Analysis_table["DirectMEonY2"]=direct_impacts_on_Y2['car']

In [24]:
# the two 'nocar's would be changed to Y1's base value name(?)
Path_Analysis_table["uxForY1"]=direct_impacts_on_Y1["U-"+'nocar']
Path_Analysis_table["MEonY1"]=direct_impacts_on_Y1['nocar']

In [25]:
# 'carcount@1' need to be changed to the Y1 name in direct_impacts_on_Y2 table
# Two 'car's need to be changed to Y2'S non-base value name
Path_Analysis_table["uY1ForY2"]=direct_impacts_on_Y2.loc['carcount@1',"U-"+'car']
Path_Analysis_table["MEofY1onY2"]=direct_impacts_on_Y2.loc['carcount@1','car']

In [26]:
Path_Analysis_table["IndirectMEonY2"]=(Path_Analysis_table["uxForY1"].apply(pd.to_numeric)+Path_Analysis_table["MEonY1"].apply(pd.to_numeric))*\
                                      (Path_Analysis_table["uY1ForY2"].apply(pd.to_numeric)+Path_Analysis_table["MEofY1onY2"].apply(pd.to_numeric))-\
                                      Path_Analysis_table["uxForY1"].apply(pd.to_numeric)*Path_Analysis_table["uY1ForY2"].apply(pd.to_numeric)

In [27]:
Path_Analysis_table["TotalMEonY2"]=Path_Analysis_table["DirectMEonY2"] + Path_Analysis_table["IndirectMEonY2"]

In [28]:
Path_Analysis_table

,Var base,Var specific,DirectMEonY2,uxForY1,MEonY1,uY1ForY2,MEofY1onY2,IndirectMEonY2,TotalMEonY2
tripcount@3,2.000000,3,-0.000198,0.492626,0.0,0.498495,0.009047,0.004457,0.004259
tripcount@0,2.000000,0,0.0,0.492626,0.0,0.498495,0.009047,0.004457,0.004457
tripcount@1,2.000000,1,0.001468,0.492626,0.0,0.498495,0.009047,0.004457,0.005925
hhsize@2,3.000000,2,0.0,0.492402,0.000702,0.498495,0.009047,0.004811,0.004811
hhsize@4,3.000000,4,0.000517,0.492402,-0.000549,0.498495,0.009047,0.004176,0.004693
hhsize@1,3.000000,1,-0.002268,0.492402,0.0,0.498495,0.009047,0.004455,0.002187
hhsize@5,3.000000,5,0.000778,0.492402,0.00037,0.498495,0.009047,0.004643,0.005421
hhincome@3,2.000000,3,-0.000655,0.492455,-0.000394,0.498495,0.009047,0.004255,0.003601
hhincome@1,2.000000,1,-0.000321,0.492455,0.001906,0.498495,0.009047,0.005423,0.005102
gender@2,1.000000,2,0.000627,0.492499,0.000261,0.498495,0.009047,0.004588,0.005215


In [29]:
Path_Analysis_table.to_csv(model_name+".csv")

# Loop through All Models

In [13]:
from ML_Path import loop_all_model

In [14]:
list_of_models = ["RF","CNB","SVM","Boost","NN"]

In [15]:
loop_all_model(list_of_models,
                   model_data,
                   RegOrdVar,
                   OrdCatVar,
                   ConVar,
                   FirstDepVar,
                   SecondDepVar,
                   col_to_var,
                   col_to_var_base,
                   test_size = 0.2)

delete gender_1
delete education_2
delete workschoolretire_0
delete urbansize_1
delete travelburden_2
delete tripcount_2
delete hhsize_3
delete hhincome_2
delete carcount_0
X+Y1=Y2 Time =
0.7300472259521484
RF Accuracy: 0.9137506160670281
[0, 1]
tripcount
hhsize
hhincome
gender
education
workschoolretire
urbansize
travelburden
carcount
modetowork
tripmile
age
Transit.Score
Walk.Index_Pop
tripmile [4.29835390161713, -4.298353901616952]
age [0.05207980291608235, -0.05207980291607884]
Transit.Score [-0.4911840391548338, 0.4911840391548345]
Walk.Index_Pop [-1.031970043168213, 1.03197004316821]
gender 2
education 3
education 1
workschoolretire 1
travelburden 3
travelburden 1
tripcount 3
tripcount 0
tripcount 1
hhsize 2
hhsize 4
hhsize 1
hhsize 5
hhincome 3
hhincome 1
carcount 1
delete gender_1
delete education_2
delete workschoolretire_0
delete urbansize_1
delete travelburden_2
delete tripcount_2
delete hhsize_3
delete hhincome_2
X+Y1=Y2 Time =
0.622368574142456
RF Accuracy: 0.9078363725973

# PDP analysis

In [14]:
# 1. PDP(calculate the marginal effect)
# 1.1 PDP value
from get_PDP import get_PDP

In [15]:
partial_dependence_rs = get_PDP(model_name,model_data,dummyVar,ConVar,DepVar,col_to_var_base,0.2)
partial_dependence_rs.to_csv("lgb_partial_dependence_rs.csv",index = False)

delete gender_1
delete education_2
delete workschoolretire_0
delete urbansize_1
delete travelburden_2
delete tripcount_2
delete hhsize_3
delete hhincome_2
delete carcount_0
X+Y1=Y2 Time =
0.5205802917480469
Boost Accuracy: 0.917693445046821
(100,)
(100,)
(70,)
(70,)
(30,)
(30,)
(51,)
(51,)
